# One At the Time (OAT) Sensitivity analysis

This is the simplest case. Also called a _local_ Sensitivity analysis. One parameter is changed by keeping all the other constant and the difference in results is compared to the change. This allows to investigate how much results are affected by the specific change in the parameter. This is good for some types of analysis, but has some problems. Main issue is that the effect of a change in the parameter might be different when other parameters assume different values...so OAT can be misleading!  This problem can only be solved with a global sensitivity analysis (next notebook)

This notebook show how to perform a simple One A Time (OAT) sensitivity analysis for a example product system of a biobased product, and calculated sensitivity ratios for each parameter.

In [1]:
# Import brightway2.5 packages
import bw2calc as bc
import bw2data as bd
import numpy as np
import pandas as pd
from scipy import stats
from lci_to_bw2 import * # import all the functions of this module

In [2]:
bd.projects.set_current('advlca25') # Still working in the same project
bd.databases

Databases dictionary with 7 object(s):
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-consequential
	exldb
	lccfg
	lcctestdb
	testbiosphere
	testdb

We start by importing data about a fictional ("dummy") product system for a biobased product.

The product system includes different activities such as the production, use, and end of life of the biobased product.

In [24]:
# Import the dummy product system

# import data from csv
mydata = pd.read_csv('ALIGNED-LCI-biobased-product-dummy.csv', header = 0, sep = ",") # using csv file avoids encoding problem
mydata.head()

# keep only the columns not needed
mydb = mydata[['Activity database','Activity code','Activity name','Activity unit','Activity type',
               'Exchange database','Exchange input','Exchange amount','Exchange unit','Exchange type',
               'Exchange uncertainty type','Exchange loc','Exchange scale','Exchange negative', 'Exchange minimum', 'Exchange maximum', 
               'Simapro name',	'Simapro unit', 'Simapro type']].copy()

mydb['Exchange uncertainty type'] = mydb['Exchange uncertainty type'].fillna(0).astype(int) # uncertainty as integers
# Note: to avoid having both nan and values in the uncertainty column I use zero as default

# Create dictionary in bw format and write database to disk. 
# Shut down all other notebooks using the same project before doing this
bw2_db = lci_to_bw2(mydb) # a function from the lci_to_bw2 module

# write database
bd.Database('ALIGNED-biob-prod-dummy').write(bw2_db)

# check what foreground activities are included
for act in bd.Database('ALIGNED-biob-prod-dummy'):
    print(act, act['code'])

14:20:06 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████████████████████████████████████| 5/5 [00:00<00:00, 13434.67it/s]

14:20:06 [info     ] Vacuuming database            


'Biobased-product-use' (year, None, None) f9eabf64-b899-40c0-9f9f-2009dbb0a0b2
'Biomass-processing' (kilogram, None, None) 403a5c32-c769-46fc-8b9a-74b8eb3c79d1
'Biobased-product-manufacturing' (kilogram, None, None) a37d149a-6508-4563-8af6-e5a39b4176df
'Biobased-product-eol' (kilogram, None, None) c8301e73-d521-4a89-998b-30b7e7751011
'Biomass-growth' (kilogram, None, None) a7d34649-9c10-4423-bac3-ecab9b43b20c


In [25]:
# More info 
myact = bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2') # Biobased-product-use
myact._data

{'name': 'Biobased-product-use',
 'unit': 'year',
 'type': 'processwithreferenceproduct',
 'database': 'ALIGNED-biob-prod-dummy',
 'code': 'f9eabf64-b899-40c0-9f9f-2009dbb0a0b2',
 'id': 158202461223297027}

We calculate a static climate impact score for the fictional biobased product, to be used for reference later on.

In [29]:
# calculation of nominal LCA score
mymethod = ('ecoinvent-3.11', 'IPCC 2021', 'climate change: fossil', 'global warming potential (GWP100)')
myact = bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2') # Biobased-product-use
functional_unit = {myact: 1}
LCA = bc.LCA(functional_unit, mymethod)
LCA.lci()
LCA.lcia()
print("The nominal Global Warming impact score is", LCA.score, bd.methods[mymethod]['unit'])

The nominal Global Warming impact score is 180.28497357367445 kg CO2-Eq


### Now perform sensitivity analysis

The procedure is in **two steps**: 

1) A simulation is performed where the initial parameter (value of each exchange) is increased by 10%. New results are calculated. 

2) Sensitivity rations are calculated for each parameter and then ranked. 


#### Step 1

Iteration through all parameters, change the technology matrix and recaltulate results.

In [30]:
mymethod = ('ecoinvent-3.11', 'IPCC 2021', 'climate change: fossil', 'global warming potential (GWP100)')
myact = bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2') # Biobased-product-use
functional_unit = {myact: 1}
LCA = bc.LCA(functional_unit, mymethod)

act = list(bd.Database('ALIGNED-biob-prod-dummy'))[0]
exc = list(act.exchanges())[0]

LCA.lci()
LCA.lcia()

In [31]:
# Iterate through all exchanges in the foreground system and change the value up by 10%
par_name = []
par_values = []
par_upper_values = []
score_values = []
OAT_values  = []

mymethod = ('ecoinvent-3.11', 'IPCC 2021', 'climate change: fossil', 'global warming potential (GWP100)')
myact = bd.Database('ALIGNED-biob-prod-dummy').get('f9eabf64-b899-40c0-9f9f-2009dbb0a0b2') # Biobased-product-use
functional_unit = {myact: 1}
LCA = bc.LCA(functional_unit, mymethod)

for act in bd.Database('ALIGNED-biob-prod-dummy'): # for all activities...
    for exc in act.exchanges(): # for all exchanges...
        LCA.lci()
        LCA.lcia()
        print("initial value", LCA.score)
        
        par_name.append((act['code'], exc['input'][1]))
        par_values.append(exc['amount'])
        par_upper_values.append(exc['amount'] * 1.1)
        score_values.append(LCA.score)

        # Get the 'id' using the activity object
        col_id = bd.Database(exc['output'][0]).get(exc['output'][1]).id
        row_id = bd.Database(exc['input'][0]).get(exc['input'][1]).id

        # Use the id and the mapping dictionaries to find matrix row and columns
        if exc['type'] == "biosphere":
            col = LCA.dicts.activity[col_id] # find column index of A matrix for the activity
            row = LCA.dicts.biosphere[row_id] # find row index of B matrix for the exchange
            
            old_bio = LCA.biosphere_matrix[row,col]
            new_bio = LCA.biosphere_matrix[row,col] * 1.1
            LCA.biosphere_matrix[row,col] = new_bio 
        else:
            col = LCA.dicts.activity[col_id] # find column index of A matrix for the activity
            row = LCA.dicts.activity[row_id] # find row index of A matrix for the exchange
            
            old_tech = LCA.technosphere_matrix[row,col]
            new_tech = LCA.technosphere_matrix[row,col] * 1.1
            LCA.technosphere_matrix[row,col] = new_tech
            
        LCA.redo_lci() # uses the new A matrix
        LCA.lcia()
        OAT_values.append(LCA.score)
        print('end value', LCA.score)
        print("---")

        # Restore original matrices, so that we can have the same inital value each iteration.
        if exc['type'] == "biosphere":
            LCA.biosphere_matrix[row,col] = old_bio
        else:
            LCA.technosphere_matrix[row,col] = old_tech

initial value 180.28497357367445
end value 171.82861834073017
---
initial value 180.28497357367445
end value 192.08629718754065
---
initial value 180.28497357367445
end value 180.2856407160469
---
initial value 180.28497357367445
end value 177.78497357367445
---
initial value 180.28497357367445
end value 163.8954305215222
---
initial value 180.28497357367445
end value 193.21753571359608
---
initial value 180.28497357367445
end value 185.3798667525958
---
initial value 180.28497357367445
end value 180.28601561219878
---
initial value 180.28497357367445
end value 177.07072066025506
---
initial value 180.28497357367445
end value 180.3324201631351
---
initial value 180.28497357367445
end value 183.77320518897503
---
initial value 180.28497357367445
end value 175.65325250192762
---
initial value 180.28497357367445
end value 180.3798667525958
---
initial value 180.28497357367445
end value 185.28497357367445
---
initial value 180.28497357367445
end value 168.5280989010185
---
initial value 18

Let's look at the results. 

Initial and increased ('upper') parameter values and initial and end values of the Global Warming Impact (GWI) scores. 

In [32]:
sr = pd.DataFrame([par_name, par_values, par_upper_values, score_values, OAT_values], 
                  index = ['par_name','par_initial', 'par_upper', 'GWI_initial', 'GWI_end']).T
sr.head()

,par_name,par_initial,par_upper,GWI_initial,GWI_end
0,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, a7d3464...",1.0,1.1,180.284974,171.828618
1,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, 7a6115b...",0.5,0.55,180.284974,192.086297
2,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, 2551612...",0.5,0.55,180.284974,180.285641
3,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, 349b29d...",-1.0,-1.1,180.284974,177.784974
4,"(f9eabf64-b899-40c0-9f9f-2009dbb0a0b2, f9eabf6...",1.0,1.1,180.284974,163.895431


#### Step 2

**Sensitivity ratios** (SR) are calculated using the formula for discrete distributions, see equation (2) in Bisinella et al. (2016) 

_Bisinella, V., Conradsen, K., Christensen, T.H., Astrup, T.F., 2016. A global approach for sparse representation of uncertainty in Life Cycle Assessments of waste management systems. International Journal of Life Cycle Assessment._ https://doi.org/10.1007/s11367-015-1014-4

In [33]:
# calcualte sensitivity ratios
sr['SR'] = ((sr['GWI_end']-sr['GWI_initial'])/sr['GWI_initial']) / ((sr['par_upper']-sr['par_initial'])/sr['par_initial']) 

sr['SR_abs'] = abs(sr['SR'])

In [34]:
sr.head()

,par_name,par_initial,par_upper,GWI_initial,GWI_end,SR,SR_abs
0,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, a7d3464...",1.0,1.1,180.284974,171.828618,-0.469055,0.469055
1,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, 7a6115b...",0.5,0.55,180.284974,192.086297,0.654593,0.654593
2,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, 2551612...",0.5,0.55,180.284974,180.285641,0.000037,0.000037
3,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, 349b29d...",-1.0,-1.1,180.284974,177.784974,-0.138669,0.138669
4,"(f9eabf64-b899-40c0-9f9f-2009dbb0a0b2, f9eabf6...",1.0,1.1,180.284974,163.895431,-0.909091,0.909091


We can rank the parameters to see the most sensitive ones. I use the absolute value of SR in this case.

In [35]:
sr_sorted = sr.sort_values('SR_abs', ascending = False, ignore_index = True).copy() # important to re-index...
sr_sorted

,par_name,par_initial,par_upper,GWI_initial,GWI_end,SR,SR_abs
0,"(f9eabf64-b899-40c0-9f9f-2009dbb0a0b2, f9eabf6...",1.0,1.1,180.284974,163.895431,-0.909091,0.909091
1,"(f9eabf64-b899-40c0-9f9f-2009dbb0a0b2, a37d149...",50.0,55.0,180.284974,193.217536,0.71734,0.71734
2,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, 7a6115b...",0.5,0.55,180.284974,192.086297,0.654593,0.654593
3,"(a37d149a-6508-4563-8af6-e5a39b4176df, a37d149...",1.0,1.1,180.284974,168.528099,-0.652127,0.652127
4,"(a37d149a-6508-4563-8af6-e5a39b4176df, a7d3464...",0.5,0.55,180.284974,189.586964,0.51596,0.51596
5,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, a7d3464...",1.0,1.1,180.284974,171.828618,-0.469055,0.469055
6,"(f9eabf64-b899-40c0-9f9f-2009dbb0a0b2, c8301e7...",50.0,55.0,180.284974,185.379867,0.282602,0.282602
7,"(c8301e73-d521-4a89-998b-30b7e7751011, 349b29d...",1.0,1.1,180.284974,185.284974,0.277339,0.277339
8,"(c8301e73-d521-4a89-998b-30b7e7751011, c8301e7...",1.0,1.1,180.284974,175.653253,-0.256911,0.256911
9,"(a37d149a-6508-4563-8af6-e5a39b4176df, 403a5c3...",0.5,0.55,180.284974,183.820652,0.196116,0.196116


What are the most sensitive parameters in plain English?

In [36]:
#scroll through the table, identify the specific exchanges.
testing = []

for i in range(0, sr_sorted.shape[0]):
    
    par = sr_sorted.iloc[i,0] 
    
    for i in bd.Database('ALIGNED-biob-prod-dummy').get(par[0]).exchanges():
        if i['input'][1] == par[1]:
            print(i)
            testing.append(str(i))

Exchange: 1.0 year 'Biobased-product-use' (year, None, None) to 'Biobased-product-use' (year, None, None)>
Exchange: 50.0 kilogram 'Biobased-product-manufacturing' (kilogram, None, None) to 'Biobased-product-use' (year, None, None)>
Exchange: 0.5 kilogram 'nutrient supply from NPK (26-15-15) fertiliser' (kilogram, RoW, None) to 'Biomass-growth' (kilogram, None, None)>
Exchange: 1.0 kilogram 'Biobased-product-manufacturing' (kilogram, None, None) to 'Biobased-product-manufacturing' (kilogram, None, None)>
Exchange: 0.5 kilogram 'Biomass-growth' (kilogram, None, None) to 'Biobased-product-manufacturing' (kilogram, None, None)>
Exchange: 1.0 kilogram 'Biomass-growth' (kilogram, None, None) to 'Biomass-growth' (kilogram, None, None)>
Exchange: 50.0 kilogram 'Biobased-product-eol' (kilogram, None, None) to 'Biobased-product-use' (year, None, None)>
Exchange: 1.0 kilogram 'Carbon dioxide, fossil' (kilogram, None, ('air',)) to 'Biobased-product-eol' (kilogram, None, None)>
Exchange: 1.0 kilog

In [37]:
sr_sorted['par_long-name'] = testing
sr_sorted

,par_name,par_initial,par_upper,GWI_initial,GWI_end,SR,SR_abs,par_long-name
0,"(f9eabf64-b899-40c0-9f9f-2009dbb0a0b2, f9eabf6...",1.0,1.1,180.284974,163.895431,-0.909091,0.909091,Exchange: 1.0 year 'Biobased-product-use' (yea...
1,"(f9eabf64-b899-40c0-9f9f-2009dbb0a0b2, a37d149...",50.0,55.0,180.284974,193.217536,0.71734,0.71734,Exchange: 50.0 kilogram 'Biobased-product-manu...
2,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, 7a6115b...",0.5,0.55,180.284974,192.086297,0.654593,0.654593,Exchange: 0.5 kilogram 'nutrient supply from N...
3,"(a37d149a-6508-4563-8af6-e5a39b4176df, a37d149...",1.0,1.1,180.284974,168.528099,-0.652127,0.652127,Exchange: 1.0 kilogram 'Biobased-product-manuf...
4,"(a37d149a-6508-4563-8af6-e5a39b4176df, a7d3464...",0.5,0.55,180.284974,189.586964,0.51596,0.51596,Exchange: 0.5 kilogram 'Biomass-growth' (kilog...
5,"(a7d34649-9c10-4423-bac3-ecab9b43b20c, a7d3464...",1.0,1.1,180.284974,171.828618,-0.469055,0.469055,Exchange: 1.0 kilogram 'Biomass-growth' (kilog...
6,"(f9eabf64-b899-40c0-9f9f-2009dbb0a0b2, c8301e7...",50.0,55.0,180.284974,185.379867,0.282602,0.282602,Exchange: 50.0 kilogram 'Biobased-product-eol'...
7,"(c8301e73-d521-4a89-998b-30b7e7751011, 349b29d...",1.0,1.1,180.284974,185.284974,0.277339,0.277339,"Exchange: 1.0 kilogram 'Carbon dioxide, fossil..."
8,"(c8301e73-d521-4a89-998b-30b7e7751011, c8301e7...",1.0,1.1,180.284974,175.653253,-0.256911,0.256911,Exchange: 1.0 kilogram 'Biobased-product-eol' ...
9,"(a37d149a-6508-4563-8af6-e5a39b4176df, 403a5c3...",0.5,0.55,180.284974,183.820652,0.196116,0.196116,Exchange: 0.5 kilogram 'Biomass-processing' (k...
